<a href="https://colab.research.google.com/github/sarithamiryala/DeepLearningLab/blob/main/Experiment7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Reshape, Conv2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.utils import to_categorical



In [2]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data by scaling and reshaping
x_train = np.expand_dims(x_train, -1).astype('float32') / 255.
x_test = np.expand_dims(x_test, -1).astype('float32') / 255.

# Convert labels to one-hot encoding
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Since MobileNetV2 requires input size of at least 32x32, we need to resize the images
x_train_resized = tf.image.resize(x_train, [32, 32])
x_test_resized = tf.image.resize(x_test, [32, 32])

# MobileNetV2 expects 3 channel images, so we need to repeat the grayscale channel twice
x_train_resized = np.repeat(x_train_resized, 3, axis=-1)
x_test_resized = np.repeat(x_test_resized, 3, axis=-1)





11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
# Load the pre-trained MobileNetV2 model without the top layer
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))

# Freeze the base_model
base_model.trainable = False

# Create a new model on top of the output of the base model
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dense(10, activation='softmax')  # 10 classes for MNIST digits
])



<ipython-input-3-c9ddf033df50>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(32, 32, 3))


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])



In [5]:
# Train the model
model.fit(
    x_train_resized, y_train,
    batch_size=128,
    epochs=5,
    validation_data=(x_test_resized, y_test)
)



Epoch 1/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 58s 112ms/step - accuracy: 0.5016 - loss: 1.5582 - val_accuracy: 0.6619 - val_loss: 0.9921
Epoch 2/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 50s 107ms/step - accuracy: 0.6721 - loss: 0.9738 - val_accuracy: 0.6825 - val_loss: 0.9281
Epoch 3/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 82s 107ms/step - accuracy: 0.6873 - loss: 0.9179 - val_accuracy: 0.6914 - val_loss: 0.9010
Epoch 4/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 80s 103ms/step - accuracy: 0.6975 - loss: 0.8922 - val_accuracy: 0.6985 - val_loss: 0.8787
Epoch 5/5
469/469 ━━━━━━━━━━━━━━━━━━━━ 51s 109ms/step - accuracy: 0.7066 - loss: 0.8603 - val_accuracy: 0.7049 - val_loss: 0.8609


In [6]:
# Evaluate the model on test data
test_loss, test_acc = model.evaluate(x_test_resized, y_test)
print('Test accuracy:', test_acc)

313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 33ms/step - accuracy: 0.6939 - loss: 0.8923
Test accuracy: 0.7049000263214111
